In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_bvp

In [12]:
# Get constants and stuff set up
rho_w = 1000 # kg/m^3 (water density)
rho_i = 900 #kg/m^3 (ice density)
g = 10 # m/s^2 (gravitational acceleration)
L = 3.3*10**5 # J/kg
K0 = 10^-24 # Pa^-3 s^-2
phi_s = 0.01 # surface slope
psi_0 = rho_w*g*phi_s 
n_prime = 0.1 # m^-1/3 s (hydraulic roughness)
f = 6.6*n_prime**2 
s_to_y = 60*60*24*365.25 

# Scales
s0 = 10*10**3 # m
Q0 = 1500 # m^3/s
m0 = Q0*psi_0/L 
S0 = (f*rho_w*g*Q0**2/psi_0)**(3/8)
N0 = (K0*rho_i*S0*L/(psi_0*Q0))**(-1/3)
t0 = rho_i*S0*L/(psi_0*Q0)
M0 = Q0/s0
h0 = 90

# Model Params
eps = s0*m0/(Q0*rho_i)
r = rho_i/rho_w
delta = N0/(s0*psi_0)

# Inputs
Q_in = 10/Q0
hL_pl1 = 1
beta_r = 0.9

tau_raw = 100000 # Pa
tau = tau_raw/N0
C = 2*10**-20 # For sliding law
alpha = C*(N0**3)*t0/s0 # For sliding law

# Need to adjust
lda = 3.2
psi = 1
M = 0.00

del_t = 0.1


In [23]:
grid_space = 100
time_space = 1000

h = np.zeros(time_space)
S = np.zeros((time_space,grid_space))
Q = np.zeros((time_space,grid_space))
N = np.zeros((time_space,grid_space))
u = np.zeros((time_space,grid_space))

S[0,:] = 5*np.ones(grid_space)
h[0] = 1/3
NL = beta_r*(1-h[0])
Nt = rho_i*g*1/N0
x = np.linspace(0,1,grid_space)
psi_var = psi*(1-3*np.exp(-20*x))


In [132]:
# initial guess
y = np.zeros((2, x.size))
y[0,:]= np.sin(x*np.pi)
y[1,:]= 0.001 + M*x

In [133]:
y

array([[0.00000000e+00, 3.17279335e-02, 6.34239197e-02, 9.50560433e-02,
        1.26592454e-01, 1.58001396e-01, 1.89251244e-01, 2.20310533e-01,
        2.51147987e-01, 2.81732557e-01, 3.12033446e-01, 3.42020143e-01,
        3.71662456e-01, 4.00930535e-01, 4.29794912e-01, 4.58226522e-01,
        4.86196736e-01, 5.13677392e-01, 5.40640817e-01, 5.67059864e-01,
        5.92907929e-01, 6.18158986e-01, 6.42787610e-01, 6.66769001e-01,
        6.90079011e-01, 7.12694171e-01, 7.34591709e-01, 7.55749574e-01,
        7.76146464e-01, 7.95761841e-01, 8.14575952e-01, 8.32569855e-01,
        8.49725430e-01, 8.66025404e-01, 8.81453363e-01, 8.95993774e-01,
        9.09631995e-01, 9.22354294e-01, 9.34147860e-01, 9.45000819e-01,
        9.54902241e-01, 9.63842159e-01, 9.71811568e-01, 9.78802446e-01,
        9.84807753e-01, 9.89821442e-01, 9.93838464e-01, 9.96854776e-01,
        9.98867339e-01, 9.99874128e-01, 9.99874128e-01, 9.98867339e-01,
        9.96854776e-01, 9.93838464e-01, 9.89821442e-01, 9.848077

In [134]:
# Setting up functions
def Nye_NQ(x,y):
    S_i_var = np.interp(x,np.linspace(0,1,grid_space),S_i)
    S83 = S_i_var**(8/3)
    psi_var_interp = np.interp(x,np.linspace(0,1,grid_space),psi_var)
    return np.vstack(([(y[1]*np.abs(y[1])/S83 - psi_var_interp)/delta], 
                      [eps*(r-1)*np.abs(y[1])**3/S83 + eps*S_i_var*y[0]**3 + M]))

In [135]:
# Setting up BCs
def bc_N(ya,yb):
    return np.array([ya[0]-NL, yb[0]-Nt])

In [136]:
# solve
S_i = S[0,:]
sol = solve_bvp(Nye_NQ, bc_N,x,y,tol=0.00000001,verbose=1)

Singular Jacobian encountered when solving the collocation system on iteration 1. 
Maximum relative residual: 1.34e+00 
Maximum boundary residual: 1.13e+14
